<a href="https://colab.research.google.com/github/adawiaan/TaskWeek7/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Adawia Ananda

NPM: 2106724883

#Predicting Sentiment of a Sentence

In [1]:
# Install the necessary libraries
!pip install transformers torch --quiet

# Import the required libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('finiteautomata/bertweet-base-sentiment-analysis')
model = AutoModelForSequenceClassification.from_pretrained('finiteautomata/bertweet-base-sentiment-analysis')

# Function to perform sentiment analysis
def analyze_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Get the model outputs
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the predicted sentiment label
    scores = outputs.logits[0].numpy()
    sentiment = np.argmax(scores)
    # Mapping sentiment label to human-readable form
    labels = ['negative', 'neutral', 'positive']
    return labels[sentiment]

# Example usage
text = "I love using this model for sentiment analysis!"
sentiment = analyze_sentiment(text)
print(f"Sentiment: {sentiment}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Sentiment: positive


In [2]:
# Example usage
text = "I hate my life"
sentiment = analyze_sentiment(text)
print(f"Sentiment: {sentiment}")

Sentiment: negative


In [3]:
# Example usage
text = "I had a burger for lunch today"
sentiment = analyze_sentiment(text)
print(f"Sentiment: {sentiment}")

Sentiment: neutral


#Checking Model's Accuracy Using a Synthetic Dataset

In [6]:
# Install the datasets library
!pip install datasets --quiet

# Import the datasets library
from datasets import load_dataset
from tqdm import tqdm

# Load the TweetEval sentiment dataset
dataset = load_dataset('tweet_eval', 'sentiment')

# Function to predict sentiment for a list of texts
def predict_sentiments(texts):
    # Tokenize and make predictions for each text
    predictions = []
    for text in tqdm(texts):
        sentiment = analyze_sentiment(text)
        predictions.append(sentiment)
    return predictions

# Map the TweetEval dataset labels to the corresponding labels used in the model
label_mapping = {0: 'negative', 1: 'neutral', 2: 'positive'}

# Evaluate the model on the test split of the TweetEval sentiment dataset
true_labels = [label_mapping[label] for label in dataset['test']['label']]
predicted_labels = predict_sentiments(dataset['test']['text'])

# Calculate accuracy
accuracy = sum(np.array(predicted_labels) == np.array(true_labels)) / len(true_labels)
print(f'Accuracy: {accuracy:.4f}')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00


README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

100%|██████████| 12284/12284 [31:02<00:00,  6.60it/s]

Accuracy: 0.7207


Akurasi dari model adalah 0.7207

#Implement Attention Transformer on Pretrained Model

In [7]:
import torch
import torch.nn as nn
from transformers import AutoModel

class AttentionTransformer(nn.Module):
    def __init__(self, pretrained_model_name='finiteautomata/bertweet-base-sentiment-analysis'):
        super(AttentionTransformer, self).__init__()
        # Load the pre-trained BERTweet model
        self.bertweet = AutoModel.from_pretrained(pretrained_model_name)
        # Attention mechanism
        self.attention = nn.Linear(self.bertweet.config.hidden_size, 1)
        # Classification layer
        self.classifier = nn.Linear(self.bertweet.config.hidden_size, 3)  # 3 classes: negative, neutral, positive
        # Softmax for output probabilities
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        # Pass through the BERTweet model
        outputs = self.bertweet(input_ids=input_ids, attention_mask=attention_mask)
        # Get the hidden states of the last layer
        hidden_states = outputs.last_hidden_state  # (batch_size, sequence_length, hidden_size)
        # Compute attention weights
        attention_weights = torch.tanh(self.attention(hidden_states))  # (batch_size, sequence_length, 1)
        attention_weights = torch.softmax(attention_weights, dim=1)  # Normalize attention weights
        # Apply attention weights to hidden states
        weighted_hidden_states = hidden_states * attention_weights  # (batch_size, sequence_length, hidden_size)
        # Aggregate the weighted hidden states
        context_vector = torch.sum(weighted_hidden_states, dim=1)  # (batch_size, hidden_size)
        # Pass the context vector through the classifier
        logits = self.classifier(context_vector)  # (batch_size, 3)
        # Output probabilities
        probs = self.softmax(logits)
        return probs

# Example usage
# Initialize the model
model_with_attention = AttentionTransformer()

# Tokenize an example input
example_text = ["I really enjoyed using this product!", "This was disappointing."]
inputs = tokenizer(example_text, return_tensors="pt", padding=True, truncation=True)

# Perform a forward pass
with torch.no_grad():
    predictions = model_with_attention(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

# Display the output probabilities
print(predictions)

Some weights of RobertaModel were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[0.3382, 0.3039, 0.3579],
        [0.3297, 0.3582, 0.3121]])


In [9]:
# Define the sentiment labels
sentiment_labels = ['negative', 'neutral', 'positive']

# Classify each sentence based on the highest probability
predicted_sentiments = [sentiment_labels[torch.argmax(pred).item()] for pred in predictions]

# Display the results
for i, (sentence, sentiment) in enumerate(zip(example_text, predicted_sentiments)):
    print(f"Sentence {i+1}: '{sentence}' => Sentiment: {sentiment}")

Sentence 1: 'I really enjoyed using this product!' => Sentiment: positive
Sentence 2: 'This was disappointing.' => Sentiment: neutral


#Checking Model's Accuracy After Using Attention Transformer Using a Synthetic Dataset

In [10]:
from sklearn.metrics import accuracy_score

# Function to evaluate model accuracy
def evaluate_model_accuracy(model, dataset, tokenizer, label_mapping):
    # Get true labels and texts
    true_labels = [label_mapping[label] for label in dataset['test']['label']]
    texts = dataset['test']['text']

    # Predict sentiment for all texts in the test set
    predicted_labels = []
    for text in tqdm(texts):
        # Tokenize the input text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        # Perform a forward pass
        with torch.no_grad():
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        # Get the predicted label
        predicted_label = sentiment_labels[torch.argmax(outputs).item()]
        predicted_labels.append(predicted_label)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    return accuracy

# Evaluate accuracy of the AttentionTransformer model
print("\nEvaluating the AttentionTransformer model...")
attention_model_accuracy = evaluate_model_accuracy(model_with_attention, dataset, tokenizer, label_mapping)

# Compare the accuracies
print(f"AttentionTransformer Model Accuracy: {attention_model_accuracy:.4f}")



Evaluating the AttentionTransformer model...


100%|██████████| 12284/12284 [31:15<00:00,  6.55it/s]


AttentionTransformer Model Accuracy: 0.2882


Akurasi dari original model adalah 0.7207
Akurasi dari model dengan Attention Transformer adalah 0.2882


Akurasi model `AttentionTransformer` yang jauh lebih rendah dibandingkan model asli kemungkinan disebabkan oleh beberapa faktor. Pertama, model `AttentionTransformer` belum melalui proses fine-tuning pada dataset tertentu, sementara model asli sudah dioptimalkan untuk analisis sentimen. Kedua, attention layer dalam model baru diinisialisasi secara acak dan belum dilatih, sehingga mungkin tidak dapat menimbang token dengan tepat untuk prediksi sentimen. Ketiga, penambahan lapisan perhatian meningkatkan kompleksitas model, yang bisa menurunkan performa jika tidak diimbangi dengan data atau pelatihan yang memadai. Untuk meningkatkan akurasi, sebaiknya lakukan fine-tuning pada seluruh model, gunakan laju pembelajaran rendah, dan eksplorasi konfigurasi perhatian lain.